<a href="https://colab.research.google.com/github/wendychen215/aop113b/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install line-bot-sdk flask pyngrok flask_ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 2.6 MB/s eta 0:00:00


In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage, StickerMessage, ImageMessage, AudioMessage, VideoMessage, LocationMessage,
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('access_token'))
handler = WebhookHandler(userdata.get('channel_secret'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

import google.generativeai as genai

genai.configure(api_key=userdata.get('apikey'))
model = genai.GenerativeModel('gemini-2.0-flash')
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        action = event.message.text
        if action == "sticker":
          reply = StickerMessage(
              package_id = "446",
              sticker_id = "1989"
          )
        elif action == "image":
          reply = ImageMessage(
              original_content_url = "https://pin.it/1fAlDSGfz",
              preview_image_url = "https://i.pinimg.com/736x/ab/3b/ae/ab3baedef97aec95f4ded73e00e85039.jpg"
          )
        elif action == "location":
          reply = LocationMessage(
              title = "西子灣小自然秘境沙灘",
              address = "804高雄市鼓山區",
              latitude = 22.634659095051482,
              longitude = 22.634659095051482
          )
        elif action == "video":
          reply = VideoMessage(
              original_content_url = "https://cdn.pixabay.com/video/2023/03/08/153821-806526710_tiny.mp4",
              preview_image_url = "https://cdn.pixabay.com/video/2023/03/08/153821-806526710_tiny.jpg"
          )
        else:
          response = ask_gemini(event.message.text)
          reply = TextMessage(text=response)

        line_bot_api.reply_message(
          ReplyMessageRequest(
              reply_token=event.reply_token,
              messages=[
                  reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://1524-34-53-50-160.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [07/May/2025 06:08:10] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 06:08:10] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 06:08:22] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 06:08:39] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 06:09:15] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 06:09:31] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 06:09:49] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 06:10:17] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 06:10:21] "POST /callback HTTP/1.1" 200 -
